<a href="https://colab.research.google.com/github/Jyoti1706/PythonDeepDive/blob/master/AssemblyWorkShop_SIMPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install simpy

In [2]:
import simpy
import random
import numpy as np
import matplotlib.pyplot as plt

### Priority Resource

In [3]:
jobs={}
stats = {'cycle_time':{1:[], 2:[]}}


def process_job(env, name, job_priority, workers):
  with workers.request(priority=job_priority) as req:
    yield req
    yield env.timeout(20)
  print(f'Job {name} with priority {job_priority} finished at {env.now:.2f}')
  jobs["name"] = env.now
  stats['cycle_time'][job_priority].append(jobs[name])



def job_arrival(env, workers):
  i = 0
  while True:
    i+=1
    yield env.timeout(random.expovariate(1/10))
    job_priority = np.random.choice([1,2], p=[0.2,0.8])
    name = i
    jobs[name] = env.now
    env.process(process_job(env, name, job_priority, workers))
    if i>=100:
      break



env = simpy.Environment()
workers = simpy.PriorityResource(env, capacity=1)
env.process(job_arrival(env, workers))
env.run()
print(f"Average Cycle Time for job 1: {np.mean(stats['cycle_time'][1])} ")
print(f"Average Cycle Time for job 2: {np.mean(stats['cycle_time'][2])}")

Job 1 with priority 2 finished at 24.66
Job 2 with priority 2 finished at 58.94
Job 3 with priority 1 finished at 78.94
Job 4 with priority 2 finished at 98.94
Job 5 with priority 2 finished at 118.94
Job 6 with priority 2 finished at 138.94
Job 7 with priority 2 finished at 158.94
Job 11 with priority 1 finished at 178.94
Job 8 with priority 2 finished at 198.94
Job 13 with priority 1 finished at 218.94
Job 14 with priority 1 finished at 238.94
Job 9 with priority 2 finished at 258.94
Job 10 with priority 2 finished at 278.94
Job 12 with priority 2 finished at 298.94
Job 15 with priority 2 finished at 318.94
Job 28 with priority 1 finished at 338.94
Job 34 with priority 1 finished at 358.94
Job 35 with priority 1 finished at 378.94
Job 37 with priority 1 finished at 398.94
Job 38 with priority 1 finished at 418.94
Job 41 with priority 1 finished at 438.94
Job 45 with priority 1 finished at 458.94
Job 16 with priority 2 finished at 478.94
Job 17 with priority 2 finished at 498.94
Job 5

### Store (Producer , Consumer)

In [4]:
veggies = ['carrot', 'potato','onion','cabbage']

def producer(env, store):
  while True:
    yield env.timeout(2)
    vegetables = random.choice(veggies)
    yield store.put(  vegetables)
    print(f'Produced {vegetables} at {env.now:.2f}')


def customer(name, env, store):
  print(f'Customer {name} started at {env.now:.2f}')
  vegetable = yield store.get()
  print(f'Customer {name}, got {vegetable} vegatable from store and leaves at {env.now:.2f}')

def customer_generator(env, store):
  i = 0
  while True:
    yield env.timeout(1)
    env.process(customer(name = i, env=env, store=store))
    i+=1

env = simpy.Environment()
store = simpy.Store(env, capacity=5)
env.process(producer(env, store))
env.process(customer_generator(env, store))
env.run(until=100)

Customer 0 started at 1.00
Customer 1 started at 2.00
Produced onion at 2.00
Customer 0, got onion vegatable from store and leaves at 2.00
Customer 2 started at 3.00
Customer 3 started at 4.00
Produced onion at 4.00
Customer 1, got onion vegatable from store and leaves at 4.00
Customer 4 started at 5.00
Customer 5 started at 6.00
Produced cabbage at 6.00
Customer 2, got cabbage vegatable from store and leaves at 6.00
Customer 6 started at 7.00
Customer 7 started at 8.00
Produced carrot at 8.00
Customer 3, got carrot vegatable from store and leaves at 8.00
Customer 8 started at 9.00
Customer 9 started at 10.00
Produced carrot at 10.00
Customer 4, got carrot vegatable from store and leaves at 10.00
Customer 10 started at 11.00
Customer 11 started at 12.00
Produced cabbage at 12.00
Customer 5, got cabbage vegatable from store and leaves at 12.00
Customer 12 started at 13.00
Customer 13 started at 14.00
Produced cabbage at 14.00
Customer 6, got cabbage vegatable from store and leaves at 14

### Store 2

In [6]:


def producer(env, store):
  while True:
    yield env.timeout(1)
    vegetable = 'Potato'
    expiry_in = env.now+random.randint(1,10)
    vegetables = simpy.PriorityItem(priority = expiry_in, item=vegetable)
    yield store.put(  vegetables)
    print(f'Produced {vegetables} at {env.now:.2f}')


def customer(name, env, store):
  print(f'Customer {name} started at {env.now:.2f}')
  vegetable = yield store.get()
  print(f'Customer {name}, got {vegetable} vegatable from store and leaves at {env.now:.2f}')

def customer_generator(env, store):
  i = 0
  while True:
    yield env.timeout(3)
    env.process(customer(name = i, env=env, store=store))
    i+=1

env = simpy.Environment()
store = simpy.PriorityStore(env, capacity=5)
env.process(producer(env, store))
env.process(customer_generator(env, store))
env.run(until=20)

Produced PriorityItem(priority=6, item='Potato') at 1.00
Produced PriorityItem(priority=12, item='Potato') at 2.00
Customer 0 started at 3.00
Customer 0, got PriorityItem(priority=6, item='Potato') vegatable from store and leaves at 3.00
Produced PriorityItem(priority=11, item='Potato') at 3.00
Produced PriorityItem(priority=13, item='Potato') at 4.00
Produced PriorityItem(priority=14, item='Potato') at 5.00
Customer 1 started at 6.00
Customer 1, got PriorityItem(priority=11, item='Potato') vegatable from store and leaves at 6.00
Produced PriorityItem(priority=13, item='Potato') at 6.00
Produced PriorityItem(priority=13, item='Potato') at 7.00
Customer 2 started at 9.00
Customer 2, got PriorityItem(priority=12, item='Potato') vegatable from store and leaves at 9.00
Produced PriorityItem(priority=16, item='Potato') at 9.00
Customer 3 started at 12.00
Customer 3, got PriorityItem(priority=13, item='Potato') vegatable from store and leaves at 12.00
Produced PriorityItem(priority=11, item=

### Filter Store

In [ ]:
veggies = ['carrot', 'potato','onion','cabbage']

def producer(env, store):
  while True:
    yield env.timeout(2)
    vegetables = random.choice(veggies)
    yield store.put(  vegetables)
    print(f'Produced {vegetables} at {env.now:.2f}')


def customer(name, env, store):
  print(f'Customer {name} started at {env.now:.2f}')
  vegetable = yield store.get()
  print(f'Customer {name}, got {vegetable} vegatable from store and leaves at {env.now:.2f}')

def customer_generator(env, store):
  i = 0
  while True:
    yield env.timeout(1)
    env.process(customer(name = i, env=env, store=store))
    i+=1

env = simpy.Environment()
store = simpy.FilterStore(env, capacity=5)
env.process(producer(env, store))
env.process(customer_generator(env, store))
env.run(until=100)